# Agents as Tools with Strands Agents


"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.



## Key Benefits and Core Principles

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task


In [3]:
import os

from strands import Agent, tool
from strands_tools import file_write


In this module we will be creating an orchestrator based multi-agent workflow. 

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

We will also explore `use_llm` which allows use to create nested agents.

## Research Agent

Lets first create a basic reasearch assistant with http_request tool. 

In [4]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [5]:
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

# Amazon Bedrock: Overview and Features

Amazon Bedrock is AWS's fully managed service that provides access to various foundation models (FMs) through a unified API. Launched in 2023, it allows developers to build and scale generative AI applications without managing the underlying infrastructure.

## Key Features

### Model Access and Integration
- **Multiple Foundation Models**: Access to models from AI21 Labs, Anthropic, Cohere, Meta, Mistral AI, Stability AI, and Amazon's own models (Amazon Titan)[1][2]
- **Unified API**: Single API for accessing different foundation models[1]
- **Serverless Experience**: No infrastructure management required[1]

### Customization and Management
- **Model Customization**: Fine-tune models with organization-specific data[1]
- **Private Model Hosting**: Deploy custom models in private environments[1]
- **Model Evaluation Tools**: Compare and evaluate different models[2]

### Security and Governance
- **Built-in Security**: VPC endpoints, encryption, 

Now we can wrap this agent as a tool. Allowing other agents to interact with it. 

####  Best Practices for Agent as Tools

When implementing the "Agents as Tools" pattern with Strands Agents:

1. Clear tool documentation: Write descriptive docstrings that explain the agent's expertise
2. Focused system prompts: Keep each specialized agent tightly focused on its domain
3. Proper response handling: Use consistent patterns to extract and format responses
4. Tool selection guidance: Give the orchestrator clear criteria for when to use each specialized agen

In [6]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

Now lets follow the best practices and create `product_recommendation_assistant`, `trip_planning_assistant`, and `orchestrator` agent.

### Product Recommendation Assistant

In [7]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [8]:
product_recommendation_assistant("Product recommendations for flying cars")

# Flying Car Product Recommendations

While true flying cars aren't widely available on the consumer market yet, I can suggest some current products that represent the closest options in development or conceptual stages:

## Current Options in Development

1. **PAL-V Liberty** - A gyrocopter/car hybrid that's completed flight certification. Requires both a driver's and pilot's license to operate. (~$300,000-600,000)
   [Source: PAL-V official website]

2. **AirCar by Klein Vision** - Recently completed a 35-minute flight between airports in Slovakia. Transforms from car to aircraft in under 3 minutes.
   [Source: Klein Vision official communications]

3. **Terrafugia Transition** - A street-legal aircraft with foldable wings, granted FAA approval for flight.
   [Source: Terrafugia company information]

4. **Aeromobil 4.0** - Slovakian flying car that can transform from car to plane in under 3 minutes.
   [Source: Aeromobil website]

## For Enthusiasts & Collectors

If you're interested

"# Flying Car Product Recommendations\n\nWhile true flying cars aren't widely available on the consumer market yet, I can suggest some current products that represent the closest options in development or conceptual stages:\n\n## Current Options in Development\n\n1. **PAL-V Liberty** - A gyrocopter/car hybrid that's completed flight certification. Requires both a driver's and pilot's license to operate. (~$300,000-600,000)\n   [Source: PAL-V official website]\n\n2. **AirCar by Klein Vision** - Recently completed a 35-minute flight between airports in Slovakia. Transforms from car to aircraft in under 3 minutes.\n   [Source: Klein Vision official communications]\n\n3. **Terrafugia Transition** - A street-legal aircraft with foldable wings, granted FAA approval for flight.\n   [Source: Terrafugia company information]\n\n4. **Aeromobil 4.0** - Slovakian flying car that can transform from car to plane in under 3 minutes.\n   [Source: Aeromobil website]\n\n## For Enthusiasts & Collectors\n\

### Trip Planning Assistant

In [9]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator Agent

In [10]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [11]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # Optional: Specify the model ID
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [12]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

I can help you find hiking boots recommendations. To provide you with the best suggestions, I'll use the product recommendation assistant and save the results to a file in your current directory.
Tool #1: product_recommendation_assistant
# Hiking Boot Recommendations

Based on your interest in quality hiking boots with a focus on durability, comfort, and terrain versatility, here are some well-regarded options:

## All-Around Performance Hiking Boots

**Salomon Quest 4 GTX**
- Excellent durability with robust leather/synthetic upper
- Exceptional ankle support and stability
- GORE-TEX waterproofing for wet conditions
- Responsive cushioning for long-distance comfort
- Versatile for various terrains including rocky, muddy, and moderate snow

**Merrell Moab 3 Mid Waterproof**
- Known as the "Mother of All Boots" for reliability
- Great value-to-performance ratio
- Comfortable right out of the box with minimal break-in
- Suitable for moderate terrain and day hiking
- Durable Vibram outsol

╔════════ File Write Operation ═════════╗
║                                       ║
║ Path: hiking_boot_recommendations.txt ║
║ Size: 1336 characters                 ║
║                                       ║
╚═══════════════════════════════════════╝

╔══════════════════════ Write Successful ══════════════════════╗
║ File written successfully to hiking_boot_recommendations.txt ║
╚══════════════════════════════════════════════════════════════╝

I've created a file with hiking boot recommendations in your current directory. The file is named "hiking_boot_recommendations.txt" and contains detailed information about several highly-rated hiking boots, including:

1. All-around performance options:
   - Salomon Quest 4 GTX
   - Merrell Moab 3 Mid Waterproof

2. Technical terrain specialists:
   - La Sportiva Nucleo High II GTX

3. Lightweight hiking options:
   - Hoka Anacapa Mid GTX

Each recommendation includes details about durability, comfort features, and what type of terrain they're best suited for. If you'd like more specific recommendations based on your hiking environment, foot shape, or other preferences, just let me know!

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

In [13]:
orchestrator.messages

[{'role': 'user',
  'content': [{'text': "I'm looking for hiking boots. Write the final response to current directory."}]},
 {'role': 'assistant',
  'content': [{'text': "I can help you find hiking boots recommendations. To provide you with the best suggestions, I'll use the product recommendation assistant and save the results to a file in your current directory."},
   {'toolUse': {'toolUseId': 'tooluse_nqAmS4q2R-SpyLz5fzSvWg',
     'name': 'product_recommendation_assistant',
     'input': {'query': "I'm looking for hiking boots. Please provide recommendations for quality hiking boots with details on durability, comfort, and suitability for different terrain types."}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_nqAmS4q2R-SpyLz5fzSvWg',
     'status': 'success',
     'content': [{'text': '# Hiking Boot Recommendations\n\nBased on your interest in quality hiking boots with a focus on durability, comfort, and terrain versatility, here are some well-regarded 

In [14]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

I'd be happy to help you plan a trip to Patagonia! I'll use the trip planning assistant to create a detailed itinerary for you. To provide the most helpful plan, it would be useful to know some details about your trip preferences, such as:

- When you're planning to visit
- How long you plan to stay
- Your interests (hiking, wildlife, photography, etc.)
- Your travel style/budget
- Any specific areas of Patagonia you want to visit

However, I can create a general Patagonia itinerary for you now based on popular destinations and experiences:
Tool #3: trip_planning_assistant
# PATAGONIA TRAVEL ITINERARY: ARGENTINA & CHILE

## BEST TIME TO VISIT
* **Peak Season (December-February)**: Ideal weather, but crowded and expensive
* **Shoulder Season (October-November & March-April)**: Good weather with fewer crowds
* **Winter (May-September)**: Limited access to many attractions, but perfect for winter sports

## 14-DAY COMPREHENSIVE ITINERARY

### DAYS 1-2: BUENOS AIRES, ARGENTINA
*Gateway to 

In [ ]:
orchestrator.messages

### Calling multiple agents 

In [ ]:
orchestrator.messages = []

In [ ]:
query = "Can you do a research on spain? Also help me plan a 7 day trip."

orchestrator(query)

Behind the scenes, the orchestrator will:
1. First call the `research_assistant`
2. Then call `trip_planning_assistant`
3. Combine these specialized responses into a cohesive answer that addresses both the queries

### Sequential Agent Communication Pattern


The agent tool can also combine multiple agents together. In this example we will provide output of `research_agent` to `summary_agent` and return the summarized response.

In [15]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

Multiple agents created successfully!

🔍 RESEARCH AGENT working on: generative Ai

# Generative AI: Comprehensive Overview

Generative AI refers to artificial intelligence systems capable of creating new content, including text, images, music, code, and more. Here's a comprehensive overview:

## Core Technologies and Models

1. **Large Language Models (LLMs)**
   - GPT (Generative Pre-trained Transformer) series by OpenAI (GPT-3, GPT-4)
   - Google's PaLM, Gemini, and LaMDA
   - Meta's LLaMA family
   - Anthropic's Claude models

2. **Image Generation Models**
   - Diffusion models: DALL-E, Midjourney, Stable Diffusion
   - GANs (Generative Adversarial Networks): StyleGAN, BigGAN

3. **Multimodal Models**
   - Can process and generate multiple types of content (text, images, audio)
   - Examples: GPT-4V, Gemini, Claude Opus

## Technical Foundations

- **Transformer Architecture**: Core architecture behind most modern LLMs
- **Diffusion Models**: Generate images by gradually denoising 

## Congrats!

You've learned how to use agents as tools in Strands Agents to create more complex agentic applications